In [1]:
from datasets.mnist import MNIST

from nn.model import Sequential

from nn.activations import relu, tanh, sigmoid, softmax, leaky_relu
from nn.layers import Dense
from nn.loss import MeanSquaredError, CategoricalCrossEntropy, CrossEntropyCost, MeanAbsoluteError
from nn.optimizer import GradientDescent
from nn.preprocessing import categorical_encoding, transform_input_data

import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import Normalizer

# import warnings
# warnings.filterwarnings("error")

In [2]:
data = MNIST(mode='full')

All required files are exist!


In [3]:
def normalize(data: np.ndarray) -> np.ndarray:
    data = data.astype('float')
    return data / 255.

In [4]:

train_data, test_data = data.dataset
X_train, y_train = train_data
X_test, y_test = test_data

print('Train:', X_train.shape, y_train.shape)
print('Test:', X_test.shape, y_test.shape)

# scaler = Normalizer()
# scaler.fit(X_train)
X_train = normalize(X_train)
X_test_ = normalize(X_test)

X_test_ = transform_input_data(X_test_)
X_train_ = transform_input_data(X_train)

y_train_ = categorical_encoding(y_train)
y_train_ = transform_input_data(y_train_)

print('Training shapes')
print('X_train:', X_train_.shape)
print('y_train:', y_train_.shape)

Train: (60000, 784) (60000,)
Test: (10000, 784) (10000,)
(60000,)
Training shapes
X_train: (60000, 784, 1)
y_train: (60000, 10, 1)


In [2]:
nn = Sequential(input_shape=(1, 784),
                layers=[
                    Dense(10, activation=softmax)

                ],
                optimizer=GradientDescent(0.1),
                loss=MeanAbsoluteError())

nn.info()

+-------------------------------------------------------------------+
|                         model: Sequential                         |
+------------+---------------+------------+------------+------------+
| Layer type | Weights shape | Bias shape | W strategy | Activation |
+------------+---------------+------------+------------+------------+
|   Dense    |   (10, 784)   |  (10, 1)   |   xavier   |  softmax   |
+------------+---------------+------------+------------+------------+
|  Optimizer | GradientDescent                                      |
+-------------------------------------------------------------------+
|     Loss   | MeanAbsoluteError                                    |
+-------------------------------------------------------------------+


In [33]:
for wl in nn.weights.layers:
    print(wl.weights.mean())

-0.004059343622087148


In [34]:
nn.fit(X_train_, y_train_, epochs=2)

<class 'numpy.ndarray'> and <class 'numpy.ndarray'> types as input data
Start training for 2 epochs


Epoch 1: 100%|██████████| 60000/60000 [00:44<00:00, 1357.86samples/s]


MeanAbsoluteError: 0.1814966242207398


Epoch 2: 100%|██████████| 60000/60000 [00:43<00:00, 1366.60samples/s]

MeanAbsoluteError: 0.17918050025873206


In [35]:
for wl in nn.weights.layers:
    print(wl.weights.mean())

-168.97338326385628


In [36]:
from sklearn.metrics import accuracy_score

def calc_accuracy(y_true: np.ndarray, x_test) -> float:
    y_hats = []
    for x_i in x_test:
        y_hat = nn.predict(x_i)
        y_hats.append(np.argmax(y_hat))
    
    acc = accuracy_score(y_true, y_hats)
    return acc, y_hats

In [37]:
acc, y1 = calc_accuracy(y_test, X_test_)
acc

0.5553

In [38]:
acc, y2 = calc_accuracy(y_train, X_train_)
acc

0.5747833333333333

In [64]:
y2

[4,
 5,
 9,
 2,
 2,
 1,
 0,
 5,
 0,
 1,
 4,
 9,
 0,
 3,
 0,
 6,
 0,
 0,
 0,
 0,
 8,
 6,
 5,
 2,
 4,
 9,
 0,
 9,
 5,
 3,
 0,
 5,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 7,
 0,
 2,
 2,
 9,
 4,
 0,
 0,
 0,
 9,
 0,
 7,
 2,
 9,
 2,
 7,
 5,
 9,
 9,
 2,
 0,
 9,
 1,
 0,
 1,
 0,
 3,
 0,
 2,
 0,
 4,
 0,
 2,
 9,
 1,
 0,
 5,
 8,
 9,
 0,
 0,
 9,
 1,
 5,
 8,
 2,
 5,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 2,
 0,
 6,
 0,
 4,
 1,
 0,
 9,
 3,
 8,
 0,
 0,
 4,
 9,
 7,
 7,
 9,
 2,
 2,
 9,
 9,
 0,
 0,
 0,
 4,
 0,
 1,
 0,
 7,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 5,
 0,
 1,
 0,
 2,
 0,
 3,
 0,
 7,
 9,
 1,
 0,
 0,
 7,
 1,
 0,
 0,
 8,
 1,
 5,
 9,
 8,
 3,
 0,
 9,
 0,
 2,
 8,
 7,
 3,
 1,
 0,
 2,
 0,
 3,
 2,
 9,
 0,
 9,
 0,
 0,
 9,
 5,
 0,
 4,
 0,
 0,
 0,
 6,
 0,
 1,
 1,
 3,
 4,
 9,
 5,
 2,
 8,
 0,
 8,
 0,
 0,
 3,
 0,
 5,
 0,
 6,
 0,
 4,
 9,
 5,
 0,
 9,
 9,
 0,
 0,
 1,
 8,
 0,
 1,
 7,
 7,
 7,
 9,
 3,
 5,
 1,
 1,
 1,
 0,
 0,
 0,
 9,
 0,
 0,
 2,
 5,
 5,
 0,
 2,
 7,
 0,
 4,
 0,
 8,
 5,
 0,
 1,
 1,
 9,
 1,
 0,
 8,


In [51]:
X_train_[0]

array([[0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.   

In [11]:
y_ = nn.predict(X_train_[4])
np.argmax(y_)

7

In [12]:
y_train_[4]

array([[0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [16]:
nn.info()

+-------------------------------------------------------------------+
|                         model: Sequential                         |
+------------+---------------+------------+------------+------------+
| Layer type | Weights shape | Bias shape | W strategy | Activation |
+------------+---------------+------------+------------+------------+
|   Dense    |   (32, 784)   |  (32, 1)   |   xavier   |    relu    |
|   Dense    |    (32, 32)   |  (32, 1)   |   xavier   |    relu    |
|   Dense    |    (10, 32)   |  (10, 1)   |   xavier   |  softmax   |
+------------+---------------+------------+------------+------------+
Optimizer: GradientDescent
loss: CategoricalCrossEntropy


In [6]:
from prettytable import PrettyTable

In [7]:
table = PrettyTable()

In [8]:
table.field_names = ['Layer type',
                     'Weights shape',
                     'Bias shape',
                     'W strategy',
                     'Activation']
table.max_width = 10
print(table)

+------------+---------------+------------+------------+------------+
| Layer type | Weights shape | Bias shape | W strategy | Activation |
+------------+---------------+------------+------------+------------+
+------------+---------------+------------+------------+------------+


In [9]:
type(table)


prettytable.prettytable.PrettyTable

In [10]:
a = list(map(lambda field: len(field)+2, table.field_names))
sum(a)

63

In [11]:
f_name = ' Optimizer '
f_value = ' GradientDescent '
n_cells = len(table.field_names)
vertical_char = '|'
bottom_char = '-'
total_line_length = sum(list(map(lambda field: len(field)+2, table.field_names))) + n_cells + 1 
opt_line = vertical_char + ' ' + f_name + vertical_char +' ' + f_value
e_space = total_line_length - len(opt_line) - 1
opt_line += ' ' * e_space
opt_line += vertical_char
print(table)
print(opt_line)
# print(vertical_char, f_name, ' ' * (max_cell_size-len(f_name)))

+------------+---------------+------------+------------+------------+
| Layer type | Weights shape | Bias shape | W strategy | Activation |
+------------+---------------+------------+------------+------------+
+------------+---------------+------------+------------+------------+
|  Optimizer |  GradientDescent                                     |


In [16]:
vertical_char = '|'
bottom_char = '-'
corner_char = '+'
max_line_len = 12

def add_vertical_row(table, row_name, row_value):
    n_cells = len(table.field_names)
    total_line_length = sum(
        list(
            map(
                lambda field: len(field)+2, table.field_names
                )
            )
        ) + n_cells + 1
    if len(row_name) + 2 < max_line_len:
        row_name = row_name.center(max_line_len, ' ')
    opt_line = vertical_char + ' ' + row_name + vertical_char +' ' + row_value
    e_space = total_line_length - len(opt_line) - 1
    opt_line += ' ' * e_space
    opt_line += vertical_char
    empty_bottom_space = total_line_length - 2
    bottom_line = corner_char + '-'*empty_bottom_space + corner_char
    return opt_line, bottom_line

In [17]:
a, b = add_vertical_row(table, 'Optimizer', 'GradientDescent')
c, d = add_vertical_row(table, 'Loss', 'CategoricalCrossEntropy')

In [18]:
print(a)
print(b)
print(c)
print(d)

|  Optimizer  | GradientDescent                                     |
+-------------------------------------------------------------------+
|     Loss    | CategoricalCrossEntropy                             |
+-------------------------------------------------------------------+


In [15]:
len('  Optimizer ')

12